In [118]:
from pathlib import Path
from typing import List, Tuple
from tempfile import TemporaryDirectory

import pandas as pd
from spacy import displacy

from meddocan.data import meddocan_zip, ArchiveFolder
from meddocan.data.containers import BratAnnotations, BratFilesPair, BratSpan
from meddocan.data.docs_iterators import GsDocs
from meddocan.language.pipeline import meddocan_pipeline
from meddocan.data import meddocan_url, meddocan_zip, ArchiveFolder
from meddocan.data.docs_iterators import BratAnnotations, GsDocs
from meddocan.data.utils import set_ents_from_brat_spans
from meddocan.data.corpus import MEDDOCAN
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from typing import Optional
from spacy.tokens import Doc

def visualize_conll03(doc: Doc, nb_lines: Optional[int] = None):
    assert doc._.is_meddocan_doc, f"The doc must be a meddocan document!"

    if nb_lines is None:
        nb_lines = float("inf")

    with TemporaryDirectory() as td:
        pth = Path(td, "file.txt")
        doc_with_brat_pair.doc._.to_connl03(pth)
        for i, line in enumerate(pth.read_text().split("\n")):
            print(line)
            if i == nb_lines:
                break


# MEDDOCAN: Anonimización aplicada al ámbito médico

## Introducción

* Medical Document Anonymization Track (track 9 of the <abbr title="Iberian Languages Evaluation Forum 2019"> [IberLEF 2019](http://ceur-ws.org/Vol-2421/)).</abbr>.

    Detectar automaticamente la información sanitaria protegida (PHI)

* ¿Como?

    Name Entity Recognition con methodos de Transfer Learning basados en los transformadores


## Dataset

* Visualización de los datos

![Figure 1: An example of MEDDOCAN annotation visualized using the BRAT annotation interface.](https://temu.bsc.es/meddocan/wp-content/uploads/2019/03/image-1-1024x922.png)

* Numero de documentos

| corpus | Train | Dev | Test |
| ------ | ----- | --- | ---- |
| Qt     | 500   | 250 | 250  |

* Distribución del tipo de entidad entre los juegos de datos

|               Tipo               | Train | Dev | Test | Total |
| :------------------------------: | :---: | :-: | :--: | :---: |
|            TERRITORIO            | 1875  | 987 | 956  | 3818  |
|              FECHAS              | 1231  | 724 | 611  | 2566  |
|      EDAD SUJETO ASISTENCIA      | 1035  | 521 | 518  | 2074  |
|     NOMBRE SUJETO ASISTENCIA     | 1009  | 503 | 502  | 2014  |
|    NOMBRE PERSONAL SANITARIO     | 1000  | 497 | 501  | 1998  |
|      SEXO SUJETO ASISTENCIA      |  925  | 455 | 461  | 1841  |
|              CALLE               |  862  | 434 | 413  | 1709  |
|               PAIS               |  713  | 347 | 363  | 1423  |
|       ID SUJETO ASISTENCIA       |  567  | 292 | 283  | 1142  |
|        CORREO ELECTRONICO        |  469  | 241 | 249  |  959  |
| ID TITULACION PERSONAL SANITARIO |  471  | 226 | 234  |  931  |
|         ID ASEGURAMIENTO         |  391  | 194 | 198  |  783  |
|             HOSPITAL             |  255  | 140 | 130  |  525  |
|   FAMILIARES SUJETO ASISTENCIA   |  243  | 92  |  81  |  416  |
|           INSTITUCION            |  98   | 72  |  67  |  237  |
|     ID CONTACTO ASISTENCIAL      |  77   | 32  |  39  |  148  |
|         NUMERO TELEFONO          |  58   | 25  |  26  |  109  |
|            PROFESION             |  24   |  4  |  9   |  37   |
|            NUMERO FAX            |  15   |  6  |  7   |  28   |
|     OTROS SUJETO ASISTENCIA      |   9   |  6  |  7   |  22   |
|           CENTRO SALUD           |   6   |  2  |  6   |  14   |
|   ID EMPLEO PERSONAL SANITARIO   |   0   |  1  |  0   |   1   |
| IDENTIF VEHICULOS NRSERIE PLACAS |   0   |  0  |  0   |   0   |
|   IDENTIF DISPOSITIVOS NRSERIE   |   0   |  0  |  0   |   0   |
|     NUMERO BENEF PLAN SALUD      |   0   |  0  |  0   |   0   |
|             URL WEB              |   0   |  0  |  0   |   0   |
|       DIREC PROT INTERNET        |   0   |  0  |  0   |   0   |
|        IDENTF BIOMETRICOS        |   0   |  0  |  0   |   0   |
|       OTRO NUMERO IDENTIF        |   0   |  0  |  0   |   0   |

* **Código**

El paquete ``meddocan.data`` proporciona clases y funciones para tratar con los conjuntos de datos originales de
meddocan en la web.

- `meddocan.data.meddocan_url` contiene el enlace url para llegar a las carpetas de datos comprimidas.

In [2]:
meddocan_url

MeddocanUrl(sample='http://temu.bsc.es/meddocan/wp-content/uploads/2019/03/sample-set.zip', train='http://temu.bsc.es/meddocan/wp-content/uploads/2019/03/train-set.zip', dev='http://temu.bsc.es/meddocan/wp-content/uploads/2019/04/dev-set-1.zip', test='http://temu.bsc.es/meddocan/wp-content/uploads/2019/05/test-set.zip', background='http://temu.bsc.es/meddocan/wp-content/uploads/2019/10/background-set.zip', base='http://temu.bsc.es/meddocan/wp-content/uploads/2019')

* `meddocan.data.meddocan_zip` obtiene el par de archivos brat para
una carpeta determinada del conjunto de datos en caché a través de su método ``brat_files``.

In [11]:
brat_files_pairs = meddocan_zip.brat_files(ArchiveFolder.train)

In [12]:
brat_files_pair = next(brat_files_pairs)

In [13]:
brat_files_pair.txt

Path('/home/wave/.meddocan/datasets/meddocan/train-set.zip', 'train/brat/S0004-06142005000500011-1.txt')

In [35]:
brat_files_pair.ann

Path('/home/wave/.meddocan/datasets/meddocan/train-set.zip', 'train/brat/S0004-06142005000500011-1.ann')

Visualizamos los datos brutos

In [37]:
brat_annotations = BratAnnotations.from_brat_files(brat_files_pair)

- El texto

In [39]:
text = brat_annotations.text
print(text)

Datos del paciente.
Nombre:  Ernesto.
Apellidos: Rivera Bueno.
NHC: 368503.
NASS: 26 63514095.
Domicilio:  Calle Miguel Benitez 90.
Localidad/ Provincia: Madrid.
CP: 28016.
Datos asistenciales.
Fecha de nacimiento: 03/03/1946.
País: España.
Edad: 70 años Sexo: H.
Fecha de Ingreso: 12/12/2016.
Médico:  Ignacio Navarro Cuéllar NºCol: 28 28 70973.
Informe clínico del paciente: Paciente de 70 años de edad, minero jubilado, sin alergias medicamentosas conocidas, que presenta como antecedentes personales: accidente laboral antiguo con fracturas vertebrales y costales; intervenido de enfermedad de Dupuytren en mano derecha y by-pass iliofemoral izquierdo; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; enolismo activo, fumador de 20 cigarrillos / día.
Es derivado desde Atención Primaria por presentar hematuria macroscópica postmiccional en una ocasión y microhematuria persistente posteriormente, con micciones normales.
En la exploración física presenta un buen estado general, 

- Las anotaciones

In [36]:
print(
    brat_files_pair
    .ann
    .root
    .open(brat_files_pair.ann.at)
    .read()
    .decode("utf-8")
)

T1	FECHAS 215 225	03/03/1946
T2	CORREO_ELECTRONICO 2421 2439	nnavcu@hotmail.com
T3	PAIS 2406 2412	España
T4	TERRITORIO 2398 2404	Madrid
T5	TERRITORIO 2392 2397	28036
T6	CALLE 2365 2391	c/ del Abedul 5-7, 2º dcha
T7	NOMBRE_PERSONAL_SANITARIO 303 326	Ignacio Navarro Cuéllar
T8	NOMBRE_PERSONAL_SANITARIO 2341 2364	Ignacio Navarro Cuéllar
T9	EDAD_SUJETO_ASISTENCIA 389 396	70 años
T10	ID_TITULACION_PERSONAL_SANITARIO 334 345	28 28 70973
T11	FECHAS 282 292	12/12/2016
T12	SEXO_SUJETO_ASISTENCIA 261 262	H
T13	EDAD_SUJETO_ASISTENCIA 247 254	70 años
T14	PAIS 233 239	España
T15	TERRITORIO 166 171	28016
T16	TERRITORIO 154 160	Madrid
T17	CALLE 107 130	Calle Miguel Benitez 90
T18	ID_ASEGURAMIENTO 82 93	26 63514095
T19	ID_SUJETO_ASISTENCIA 68 74	368503
T20	NOMBRE_SUJETO_ASISTENCIA 49 61	Rivera Bueno
T21	NOMBRE_SUJETO_ASISTENCIA 29 36	Ernesto



- Las anotaciones

In [40]:
brat_spans = brat_annotations.brat_spans; brat_spans

[BratSpan(id='T1', entity_type='FECHAS', start=215, end=225, text='03/03/1946'),
 BratSpan(id='T2', entity_type='CORREO_ELECTRONICO', start=2421, end=2439, text='nnavcu@hotmail.com'),
 BratSpan(id='T3', entity_type='PAIS', start=2406, end=2412, text='España'),
 BratSpan(id='T4', entity_type='TERRITORIO', start=2398, end=2404, text='Madrid'),
 BratSpan(id='T5', entity_type='TERRITORIO', start=2392, end=2397, text='28036'),
 BratSpan(id='T6', entity_type='CALLE', start=2365, end=2391, text='c/ del Abedul 5-7, 2º dcha'),
 BratSpan(id='T7', entity_type='NOMBRE_PERSONAL_SANITARIO', start=303, end=326, text='Ignacio Navarro Cuéllar'),
 BratSpan(id='T8', entity_type='NOMBRE_PERSONAL_SANITARIO', start=2341, end=2364, text='Ignacio Navarro Cuéllar'),
 BratSpan(id='T9', entity_type='EDAD_SUJETO_ASISTENCIA', start=389, end=396, text='70 años'),
 BratSpan(id='T10', entity_type='ID_TITULACION_PERSONAL_SANITARIO', start=334, end=345, text='28 28 70973'),
 BratSpan(id='T11', entity_type='FECHAS', sta

### Preprocessamiento

Para empezar llamamos nuestro pipeline ``meddocan.language.pipeline.meddocan_pipeline`` hecho con la libreria [spaCy](https://spacy.io/) y miramos sus componentes.

In [42]:
nlp = meddocan_pipeline()
pd.DataFrame(nlp.pipe_names, columns=["componentes"]).T

,0,1,2,3
componentes,missaligned_splitter,line_sentencizer,predictor,write_methods


Vamos a hacer pasar nuestro texto de ejemplo por el ``meddocan_pipeline`` y ver el efecto de cada componente.

In [43]:
doc = nlp(text)

#### Tokenización

In [57]:
[token.orth_ for token in doc[:10]]

['Datos', 'del', 'paciente', '.', '\n', 'Nombre', ':', ' ', 'Ernesto', '.']

#### Casos especiales

In [58]:
[token for token in nlp("DREnric Lopez")]

[DR, Enric, Lopez]

#### Partición del documento en párafos

In [75]:
pd.set_option('display.max_colwidth', 100)

df = pd.DataFrame([sent.text for sent in doc.sents], columns=["párafo"]);
df.style.set_properties(**{'text-align': 'left'})


,párafo
0,Datos del paciente.
1,Nombre: Ernesto.
2,Apellidos: Rivera Bueno.
3,NHC: 368503.
4,NASS: 26 63514095.
5,Domicilio: Calle Miguel Benitez 90.
6,Localidad/ Provincia: Madrid.
7,CP: 28016.
8,Datos asistenciales.
9,Fecha de nacimiento: 03/03/1946.


#### predictor y entidades

En una primera fase no tenemos modelos con el cual hacer predicciones.
Entonces añadimos la entidades a nuestro objeto ``doc`` usando la function ``meddocan.data.utils.set_ents_from_brat_spans``.

In [19]:
doc = set_ents_from_brat_spans(doc, brat_spans=brat_spans)
displacy.render(doc[:100], style="ent")

#### Write methods

Para la evaluación, se puede escribir los datos al formato **Brat**

In [20]:
with TemporaryDirectory() as td:
    pth = Path(td, "file.txt")
    doc._.to_ann(pth)
    for i, line in enumerate(pth.read_text().split("\n")):
        print(line)
        if i > 10:
            break

T_0	NOMBRE_SUJETO_ASISTENCIA 29 36	Ernesto
T_1	NOMBRE_SUJETO_ASISTENCIA 49 61	Rivera Bueno
T_2	ID_SUJETO_ASISTENCIA 68 74	368503
T_3	ID_ASEGURAMIENTO 82 93	26 63514095
T_4	CALLE 107 130	Calle Miguel Benitez 90
T_5	TERRITORIO 154 160	Madrid
T_6	TERRITORIO 166 171	28016
T_7	FECHAS 215 225	03/03/1946
T_8	PAIS 233 239	España
T_9	EDAD_SUJETO_ASISTENCIA 247 254	70 años
T_10	SEXO_SUJETO_ASISTENCIA 261 262	H
T_11	FECHAS 282 292	12/12/2016


Para el entrenamiento, los datos se escriben al formato **conll03**.

In [21]:
with TemporaryDirectory() as td:
    pth = Path(td, "file.txt")
    doc._.to_connl03(pth)
    for i, line in enumerate(pth.read_text().split("\n")):
        print(line)
        if i > 10:
            break

Datos O
del O
paciente O
. O

Nombre O
: O
Ernesto B-NOMBRE_SUJETO_ASISTENCIA
. O

Apellidos O
: O


## Entrenamiento con la librería [Flair](https://github.com/flairNLP/flair)

### Creación del dataset ``MEDDOCAN``

Los distinctos conjunto de datos

In [79]:
pd.DataFrame([[folder.value for folder in ArchiveFolder]], index=["sets"])

,0,1,2,3,4
sets,train,dev,test,sample,background


Solo se usan los sets de *train*, *dev* y *test*.

Utilizamos el objecto ``meddocan.data.docs_iterators.GsDocs`` que nos permitte obtener objectos ``Doc`` usando el ``meddocan_pipeline`` para cada uno de los conjuntos de datos.

In [129]:
gs_docs = GsDocs(ArchiveFolder.train)
docs_with_brat_pair = iter(gs_docs)
doc_with_brat_pair = next(docs_with_brat_pair)

``doc_with_brat_pair`` es una ``NameTuple`` que associa cada ``BratFilesPair`` con el objeto ``Doc`` corespondiente.

In [130]:
doc_with_brat_pair.brat_files_pair

BratFilesPair(ann=Path('/home/wave/.meddocan/datasets/meddocan/train-set.zip', 'train/brat/S0004-06142005000500011-1.ann'), txt=Path('/home/wave/.meddocan/datasets/meddocan/train-set.zip', 'train/brat/S0004-06142005000500011-1.txt'))

In [95]:
doc_with_brat_pair.doc[:10]

Datos del paciente.
Nombre:  Ernesto.

In [128]:
visualize_conll03(doc_with_brat_pair.doc, nb_lines=8)

Datos O
del O
paciente O
. O

Nombre O
: O
Ernesto B-NOMBRE_SUJETO_ASISTENCIA
. O


``doc_with_brat_pair`` ayuda a crear el ``MEDDOCAN`` corpus que hereda de ``flair.datasets.ColumnCorpus``.

El ``MEDDOCAN`` corpus esta creado a partir de ficheros temporales que contienen el texto asi como los offsets al formato **connl03**.

In [126]:
corpus = MEDDOCAN(sentences=True, in_memory=True, document_separator_token="-DOCSTART-")

2023-03-01 12:05:44,870 Reading data from /tmp/tmpww59dhkd
2023-03-01 12:05:44,870 Train: /tmp/tmpww59dhkd/train
2023-03-01 12:05:44,871 Dev: /tmp/tmpww59dhkd/dev
2023-03-01 12:05:44,871 Test: /tmp/tmpww59dhkd/test


In [127]:
print(corpus)

Corpus: 10811 train + 5518 dev + 5405 test sentences


Ahora podemos pasar al entrenamiento con **Flair**.

### Entrenamiento

In [5]:
from flair.data import Corpus
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

from meddocan.data.corpus import MEDDOCAN

1. Obtener el corpus

In [136]:
corpus: Corpus = MEDDOCAN(sentences=True, document_separator_token="-DOCSTART-")

2023-03-01 12:17:14,415 Reading data from /tmp/tmp0tuo2hau
2023-03-01 12:17:14,415 Train: /tmp/tmp0tuo2hau/train
2023-03-01 12:17:14,416 Dev: /tmp/tmp0tuo2hau/dev
2023-03-01 12:17:14,417 Test: /tmp/tmp0tuo2hau/test


In [137]:
print(corpus)

Corpus: 10811 train + 5518 dev + 5405 test sentences


2. ¿Que label queremos predecir?

In [139]:
label_type = 'ner'

3. Crear el diccionario de labels a partir del corpus

In [141]:
label_dict = corpus.make_label_dictionary(label_type=label_type)

2023-03-01 12:18:48,101 Computing label dictionary. Progress:


10811it [00:00, 41822.57it/s]

2023-03-01 12:18:48,382 Dictionary created for label 'ner' with 22 values: TERRITORIO (seen 1875 times), FECHAS (seen 1231 times), EDAD_SUJETO_ASISTENCIA (seen 1035 times), NOMBRE_SUJETO_ASISTENCIA (seen 1009 times), NOMBRE_PERSONAL_SANITARIO (seen 1000 times), SEXO_SUJETO_ASISTENCIA (seen 925 times), CALLE (seen 862 times), PAIS (seen 713 times), ID_SUJETO_ASISTENCIA (seen 567 times), ID_TITULACION_PERSONAL_SANITARIO (seen 471 times), CORREO_ELECTRONICO (seen 469 times), ID_ASEGURAMIENTO (seen 391 times), HOSPITAL (seen 255 times), FAMILIARES_SUJETO_ASISTENCIA (seen 243 times), INSTITUCION (seen 98 times), ID_CONTACTO_ASISTENCIAL (seen 77 times), NUMERO_TELEFONO (seen 58 times), PROFESION (seen 24 times), NUMERO_FAX (seen 15 times), OTROS_SUJETO_ASISTENCIA (seen 9 times)


In [142]:
print(label_dict)

Dictionary with 22 tags: <unk>, TERRITORIO, FECHAS, EDAD_SUJETO_ASISTENCIA, NOMBRE_SUJETO_ASISTENCIA, NOMBRE_PERSONAL_SANITARIO, SEXO_SUJETO_ASISTENCIA, CALLE, PAIS, ID_SUJETO_ASISTENCIA, ID_TITULACION_PERSONAL_SANITARIO, CORREO_ELECTRONICO, ID_ASEGURAMIENTO, HOSPITAL, FAMILIARES_SUJETO_ASISTENCIA, INSTITUCION, ID_CONTACTO_ASISTENCIAL, NUMERO_TELEFONO, PROFESION, NUMERO_FAX, OTROS_SUJETO_ASISTENCIA, CENTRO_SALUD


4. Inicializar los embeddings generados por el transformador utilizando el contexto

In [143]:
embeddings = TransformerWordEmbeddings(
    model='dccuchile/bert-base-spanish-wwm-cased',
    layers="-1",
    subtoken_pooling="first",
    fine_tune=True,
    use_context=True,
)

In [145]:
embeddings

TransformerWordEmbeddings(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

5. Inicializar etiquedator simple (no CRF, no RNN, no reprojección)

In [4]:
tagger = SequenceTagger(
    hidden_size=256,
    embeddings=embeddings,
    tag_dictionary=label_dict,
    tag_type='ner',
    use_crf=False,
    use_rnn=False,
    reproject_embeddings=False,
)

NameError: name 'embeddings' is not defined

6. Initializar el trainer

In [3]:
trainer = ModelTrainer(tagger, corpus)

NameError: name 'tagger' is not defined

7. Ejecutar el fine-tuning

```python
trainer.fine_tune(
    'experiments/meddocan',
    learning_rate=5.0e-6,
    mini_batch_size=4,
    epoch=0
  )
```

```python
from flair.data import Corpus
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

from meddocan.data.corpus import MEDDOCAN


corpus: Corpus = MEDDOCAN(  # 1. Obtener el corpus
    sentences=True, document_separator_token="-DOCSTART-"
)


label_type = 'ner'  # 2. Que label se quiere predecir?


label_dict = corpus.make_label_dictionary(
    label_type=label_type, add_unk=False,  # 3. Crear el diccionario de labels a partir del corpus
)
print(label_dict)

embeddings = TransformerWordEmbeddings(  # 4. Inicializar los embeddings generados por el transformador utilizando el contexto
    model='dccuchile/bert-base-spanish-wwm-cased',
    layers="-1",
    subtoken_pooling="first",
    fine_tune=True,
    use_context=True,
)


tagger = SequenceTagger(  # 5. Inicializar etiquedator simple (no CRF, no RNN, no reprojección)
    hidden_size=256,
    embeddings=embeddings,
    tag_dictionary=label_dict,
    tag_type='ner',
    use_crf=False,
    use_rnn=False,
    reproject_embeddings=False,
)

trainer = ModelTrainer(tagger, corpus)  # 6. Initializar el trainer

trainer.fine_tune(  # 7. Ejecutar el fine-tuning
    'experiments/meddocan',
    learning_rate=5.0e-6,
    mini_batch_size=4,
    epoch=0
  )
```

## Inferencia

Se utiliza el ``meddocan_pipeline`` con un modelo entreando con Flair.

In [47]:
nlp = meddocan_pipeline("flair/ner-english-fast")
sys = nlp(doc.text);

2023-02-27 12:30:14,729 loading file /home/wave/.meddocan/models/ner-english-fast/4c58e7191ff952c030b82db25b3694b58800b0e722ff15427f527e1631ed6142.e13c7c4664ffe2bbfa8f1f5375bd0dced866b8c1dd7ff89a6d705518abf0a611
2023-02-27 12:30:16,367 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [51]:
displacy.render(sys[:100], style="ent")

### Transfert learning

Comparación entre el aprendizaje supervisado tradicional (izquierda) y el transfer learning (derecha)

![transfer-learning](../cdti/figures/transformers-1.png)

### Flert

![Flert](../cdti/figures/flert-1.png)

* **baseline**

    * Flair + LSTM +CRF

* **2 enfoques para el NER basados en Transformers**

    * Finetuning + linear transformation
    * Caractéristica + LSTM + CRF



* Entrenamiento: **Flair + LSTM-CRF**

![training flair](../cdti/figures/flair-1.png)

    * procedimiento de entrenamiento


|     Parameter      |       Value       |
| :----------------: | :---------------: |
|   Learning rate    |        0.1        |
|  Mini Batch size   |         4         |
|     Max epochs     |        150        |
|     Optimizer      |        SGD        |
|     Scheduler      | Anneal On Plateau |

* Entrenamiento: **Método de ajuste fino**

 Agrupación de subpalabras para crear representaciones a nivel de tokens que luego se pasan a la capa lineal final
![training finetuning](../cdti/figures/flert-2.png)

    * BETO

|     Parameter      |              Value              |
| :----------------: | :-----------------------------: |
| Transformer layers |              last               |
|   Learning rate    |              5e-6               |
|  Mini Batch size   |                4                |
|     Max epochs     |               150               |
|     Optimizer      |              AdamW              |
|     Scheduler      | Linear Warmup With Linear Decay |
|       Warmup       |               0.1               |
|  Subword pooling   |              first              |

    * XLM RoBERTa Large

|     Parameter      |              Value              |
| :----------------: | :-----------------------------: |
| Transformer layers |              last               |
|   Learning rate    |              5e-6               |
|  Mini Batch size   |                4                |
|     Max epochs     |               40                |
|     Optimizer      |              AdamW              |
|     Scheduler      | Linear Warmup With Linear Decay |
|       Warmup       |               0.1               |
|  Subword pooling   |              first              |

* Entrenamiento: **Método basado en características**

![training feature based](../cdti/figures/flert-3.png)

    * Procedimiento de entrenamiento

|     Parameter      |       Value       |
| :----------------: | :---------------: |
|   Learning rate    |        0.1        |
|  Mini Batch size   |         4         |
|     Max epochs     |        150        |
|     Optimizer      |        SGD        |
|     Scheduler      | Anneal On Plateau |

In [33]:
!tensorboard --logdir ../experiments

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

I0227 11:13:45.097182 140381710694144 plugin.py:429] Monitor runs begin
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


* Entrenamiento: Método basado en características

### Entrenamiento

In [ ]:
!cd .. && bash scripts/test-cov.sh

## Evaluación

In [ ]:
gs_docs = GsDocs(ArchiveFolder.train)
docs_with_brat_pair = iter(gs_docs)
doc_with_brat_pair = next(docs_with_brat_pair)